In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install tensorflow

In [ ]:
!pip install azure-storage-blob

In [ ]:
!pip install contractions

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter
from azure.storage.blob import BlobServiceClient
import os
import requests
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import shutil
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions
from pydantic import BaseModel

In [ ]:
# Télécharger les ressources nécessaires pour la lemmatisation
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Définir les stopwords
stop_words = set(stopwords.words('english'))

# Initialiser le stemmer
stemmer = PorterStemmer()

##### fonctions

In [ ]:
def nettoyer_texte(texte):
    """
    Nettoie le texte en supprimant les URL, les mentions, la ponctuation, les chiffres,
    les stop words et en développant les contractions. Ajoute les hashtags extraits
    à la fin du texte.

    Args:
        texte (str): Texte à nettoyer.

    Returns:
        str: Texte nettoyé.
    """
    # Convertir en minuscules
    texte = texte.lower()
    # Développer les contractions courantes
    texte = contractions.fix(texte)
    # Retirer les URL
    texte = re.sub(r"http\S+|www\S+|https\S+", '', texte, flags=re.MULTILINE)
    # Retirer les mentions d'utilisateurs
    texte = re.sub(r'@\w+', '', texte)
    # Extraire et conserver les hashtags
    hashtags = re.findall(r'#(\w+)', texte)
    texte = re.sub(r'#\w+', '', texte)
    # Retirer les caractères spéciaux et la ponctuation
    texte = re.sub(r'[^\w\s]', '', texte)
    # Retirer les chiffres
    texte = re.sub(r'\d+', '', texte)
    # Retirer les stop words
    texte = ' '.join([word for word in texte.split() if word not in stop_words])
    # Retirer les espaces multiples et superflus
    texte = re.sub(r'\s+', ' ', texte).strip()
    # Ajouter les hashtags extraits à la fin du texte
    if hashtags:
        texte = f"{texte} {' '.join(hashtags)}"

    return texte

In [ ]:
# Fonction de stemmatisation
def stemmer_texte(texte):
    tokens = nltk.word_tokenize(texte)
    return ' '.join([stemmer.stem(token) for token in tokens])

In [ ]:
# Calculer la moyenne des embeddings Word2Vec pour chaque tweet
def get_avg_word2vec_embeddings(text_data, model, vector_size):
    """
    Calcule la moyenne des embeddings Word2Vec pour chaque tweet.

    Args:
        text_data (iterable): Liste de tweets sous forme de listes de mots.
        model (Word2Vec): Modèle Word2Vec entraîné.
        vector_size (int): Taille des vecteurs Word2Vec.

    Returns:
        np.array: Tableau contenant les embeddings moyens pour chaque tweet.
    """
    embeddings = []
    for tweet in text_data:
        # Extraire les vecteurs de mots et calculer la moyenne
        word_vectors = [model.wv[word] for word in tweet if word in model.wv]
        if word_vectors:
            avg_vector = np.mean(word_vectors, axis=0)
        else:
            avg_vector = np.zeros(vector_size)  # Vecteur nul si aucun mot n'est présent dans Word2Vec
        embeddings.append(avg_vector)
    return np.array(embeddings)

##### reste

In [3]:
# Montée de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Chargement du fichier CSV dans un DataFrame
df = pd.read_csv('/content/drive/My Drive/projet_7/cleaned_tweets6.csv', encoding='latin1', sep=";")

In [ ]:
with open('/content/drive/My Drive/projet_7/token.txt', 'r') as file:
    hf_token = file.read().strip()

from huggingface_hub import login
login(hf_token)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2o7fRSSTsKkRHY1jl4VoX9qS7AR_5TCYPvixQ8rv7g5PqJp8t")

In [ ]:
# Chemin temporaire où télécharger le modèle sur Heroku
local_model_path = "/tmp/lstm_model_quantized.tflite"

# URL brute du modèle sur GitHub (remplace par ton URL)
model_url = "https://github.com/SamLouis1980/P7---Analyse-de-Sentiments/raw/main/lstm_model_quantized.tflite"

# Ton token d'accès personnel GitHub (remplace par ton token)
token = "ton_token_github"  # Remplace par ton token d'accès

# Entêtes pour l'authentification
headers = {
    "Authorization": f"token {token}"
}

# Télécharger le modèle depuis GitHub avec authentification
response = requests.get(model_url, headers=headers)

# Vérifie que le téléchargement a réussi
if response.status_code == 200:
    with open(local_model_path, 'wb') as f:
        f.write(response.content)
    print("Modèle téléchargé avec succès depuis GitHub !")
else:
    print(f"Erreur de téléchargement du modèle. Code HTTP : {response.status_code}")

# Charger le modèle TensorFlow Lite depuis le répertoire temporaire
interpreter = Interpreter(model_path=local_model_path)
interpreter.allocate_tensors()

Modèle téléchargé avec succès depuis GitHub !


In [ ]:
def download_model_from_azure():
    connect_str = "DefaultEndpointsProtocol=https;AccountName=analysedesenti6317520947;AccountKey=qImI/TRE2rCba9jbk4rstHzjYcE/OAU7IdXONkihT/6hKq1d3tyAuxgkgoUpxGIa0Ghw5uBTJJOX+ASt9z1BUw==;EndpointSuffix=core.windows.net"
    container_name = "projet7"
    blob_name = "word2vec_tweets.model"
    download_file_path = "word2vec_tweets.model"  # Le chemin où le modèle sera stocké localement

    # Créer un client BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Créer un client de conteneur
    container_client = blob_service_client.get_container_client(container_name)

    # Télécharger le fichier
    blob_client = container_client.get_blob_client(blob_name)
    with open(download_file_path, "wb") as file:
        file.write(blob_client.download_blob().readall())

    print(f"Le modèle a été téléchargé avec succès dans {download_file_path}")

    # Charger le modèle Word2Vec après l'avoir téléchargé
    word2vec_model = Word2Vec.load(download_file_path)
    print("Modèle Word2Vec chargé avec succès.")

    return word2vec_model

In [ ]:
# Appeler la fonction pour télécharger et charger le modèle
word2vec_model = download_model_from_azure()  # Appel à la fonction

Le modèle a été téléchargé avec succès dans word2vec_tweets.model
Modèle Word2Vec chargé avec succès.


In [ ]:
# Obtenir les détails des tensors pour comprendre comment passer les données
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Fonction pour préparer l'entrée pour TensorFlow Lite
def prepare_input(text):
    cleaned_text = nettoyer_texte(text)
    stemmed_text = stemmer_texte(cleaned_text)
    tokenized_text = stemmed_text.split()

    # Obtenir les embeddings Word2Vec
    word2vec_vector = get_avg_word2vec_embeddings([tokenized_text], word2vec_model, word2vec_model.vector_size)

    # Reformater l'entrée pour ajouter une dimension batch_size = 1
    # Cela suppose que le modèle attend un vecteur de la forme (batch_size, sequence_length, embedding_dim)
    word2vec_vector = np.expand_dims(word2vec_vector, axis=0)  # Ajouter la dimension batch_size
    return np.array(word2vec_vector, dtype=np.float32)

# Fonction de prédiction avec TensorFlow Lite
def predict_with_tflite(text):
    input_data = prepare_input(text)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Récupérer la sortie (probabilité) correctement
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Si output_data est un tableau NumPy, on extrait la probabilité de la première position
    probability = output_data[0][0]  # Accès à la probabilité dans le tableau NumPy

    # Déterminer la classe en fonction du seuil (par exemple, 0.5)
    prediction = 1 if probability > 0.5 else 0

    # Convertir la prédiction en texte explicite (positif ou négatif)
    prediction_label = "positive" if prediction == 1 else "negative"

    # Retourner directement les résultats sous forme de dictionnaire
    return {
        "probability": float(probability),  # Assurez-vous de retourner un type natif (float) pour JSON
        "prediction": prediction,           # La prédiction (0 ou 1)
        "prediction_label": prediction_label  # L'étiquette (positive ou negative)
    }

In [ ]:
# Définir un modèle pour la requête POST (données reçues)
class TextRequest(BaseModel):
    text: str

# Appliquer nest_asyncio pour éviter les erreurs de boucle d'événements dans Colab
nest_asyncio.apply()

# Créer l'API FastAPI
app = FastAPI()

# Modèle de demande pour la route de prédiction
class PredictionResponse(BaseModel):
    probability: float
    prediction: int
    prediction_label: str

# Route POST pour la prédiction
@app.post("/predict", response_model=PredictionResponse)
async def predict(request: TextRequest):
    text = request.text
    prediction = predict_with_tflite(text)  # Appeler la fonction de prédiction
    return prediction  # Retourner directement la réponse de la prédiction

# Route GET
@app.get("/")
async def root():
    return {"message": "Hello World"}

# Lancer l'API avec ngrok pour exposer à internet
if __name__ == "__main__":
    '''port = int(os.getenv("PORT", 8000))  # Utilise le port spécifié par Azure ou 8000 par défaut
    uvicorn.run(app, host="0.0.0.0", port=port)'''

    # Tunnel vers ngrok pour accéder à l'API à distance
    '''ngrok.set_auth_token("2o7fRSSTsKkRHY1jl4VoX9qS7AR_5TCYPvixQ8rv7g5PqJp8t")
    public_url = ngrok.connect(8000)
    print("FastAPI app is running at:", public_url)'''

    # Lancer l'API FastAPI
    uvicorn.run(app, host="0.0.0.0", port=8080)

FastAPI app is running at: NgrokTunnel: "https://62de-35-221-208-28.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [2048]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     93.27.218.137:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     93.27.218.137:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     93.27.218.137:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2048]


In [ ]:
!git commit -m "commit depuis Google Colab"
!git remote add origin https://github.com/SamLouis1980/P7---Analyse-de-Sentiments.git  # Remplace par l'URL de ton dépôt GitHub
!git push -u origin model  # Pousse vers la branche principale (master)